<a href="https://colab.research.google.com/github/stefan-gietl/heat-exchangers/blob/main/2_Nachrechnung_Waermedurchgang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wärmedurchgang WÜT
- 1 Imports
- 2 Ausgangsdaten
  - 2.1 Luftseite
  - 2.2. Rauchgasseite
- 3 Nachrechnung
  - 3.1 Luftseite
  - 3.2 Rauchgasseite
- 4 Ausgaben


### 1 Imports

In [ ]:
!pip install pyfluids
import math
from CoolProp.CoolProp import PropsSI

## 2 Ausgangsdaten


In [ ]:
DruckN = 101325  # Pa
Druck_Heiss = 101575  # Pa
Druck_Kalt = 101575  # Pa

rho_norm = PropsSI("D", "T", 273.15, "P", DruckN, "air")

Re_lam = 2300
Re_turb = 10000

##### 2.1 Luftseite

In [ ]:
Tein_Kalt = 25  # °C
Taus_Kalt = 251  # °C
Tmittel_Kalt = (Tein_Kalt + Taus_Kalt) / 2 + 273.15  # K
V_N_Kalt = 854  # Nm^3/h
rho_LM = PropsSI("D", "T", Tmittel_Kalt, "P", Druck_Kalt, "air")
V_L_real = V_N_Kalt * rho_norm / rho_LM  # m^3/h
nu_L = PropsSI("V", "T", Tmittel_Kalt, "P", Druck_Kalt, "air") / rho_LM
lambda_L = PropsSI("L", "T", Tmittel_Kalt, "P", Druck_Kalt, "air")
Pr_L = PropsSI("PRANDTL", "T", Tmittel_Kalt, "P", Druck_Kalt, "air")

##### 2.2 Rauchgasseite

In [ ]:
Tein_Heiss = 450  # °C
Taus_Heiss = 113  # °C
Tmittel_Heiss = (Tein_Heiss + Taus_Heiss) / 2 + 273.15  # K
V_N_Heiss = 559  # Nm^3/h
rho_RM = PropsSI("D", "T", Tmittel_Heiss, "P", Druck_Heiss, "air")
V_R_real = V_N_Heiss * rho_norm / rho_RM  # m^3/h
nu_R = PropsSI("V", "T", Tmittel_Heiss, "P", Druck_Heiss, "air") / rho_RM
lambda_R = PropsSI("L", "T", Tmittel_Heiss, "P", Druck_Heiss, "air")
Pr_R = PropsSI("PRANDTL", "T", Tmittel_Heiss, "P", Druck_Heiss, "air")

### 3 Nachrechnung
##### 3.1 Luftseite

In [ ]:
Kanalweite_L = 3.5  # mm
Kanalhoehe_L = 226  # mm
Kanallaenge_L = 1168  # mm
N_Blockreihen_L = 1  # -
N_Kanaele_L = 57  # -
N_Ebenen_L = 1  # -
Umittel_L = V_L_real / 3600 / (Kanalweite_L * 0.001) / (Kanalhoehe_L * 0.001) / N_Kanaele_L / N_Ebenen_L  # m/s
D_hydr_L = 2 * Kanalweite_L  # mm
Re_L = Umittel_L * D_hydr_L / 1000 / nu_L  # -

if Re_L < Re_lam:
    print("Re-Zahl Luftseite im laminaren Bereich")
    Gz_LRe = Re_L * Pr_L * D_hydr_L / Kanallaenge_L
    Nu1_LRe = 7.54
    Nu2_LRe = 1.849 * Gz_LRe**(1 / 3)
    Nu3_LRe = (2 / (1 + 22 * Pr_L))**(1 / 6) * Gz_LRe**(1 / 2)
    Nu_L = (Nu1_LRe**3 + Nu2_LRe**3 + Nu3_LRe**3)**(1 / 3)
elif Re_L < Re_turb:
    print("Re-Zahl Luftseite im Übergangsbereich")
    Gz_L2300 = Re_lam * Pr_L * D_hydr_L / Kanallaenge_L
    Nu1_L2300 = 7.54
    Nu2_L2300 = 1.849 * Gz_L2300**(1 / 3)
    Nu3_L2300 = (2 / (1 + 22 * Pr_L))**(1 / 6) * Gz_L2300**(1 / 2)
    Nu_L2300 = (Nu1_L2300**3 + Nu2_L2300**3 + Nu3_L2300**3)**(1 / 3)

    xi_L10000 = (1.8 * math.log10(Re_turb) - 1.5)**(-2)
    Nu_L10000 = ((xi_L10000 / 8) * Re_turb * Pr_L) / (1 + 12.7 * ((xi_L10000 / 8)**(1 / 2)) * (Pr_L**(2 / 3) - 1))
    gamma_L = (Re_L - Re_lam) / (Re_turb - Re_lam)
    Nu_L = (1 - gamma_L) * Nu_L2300 + gamma_L * Nu_L10000
else:
    print("Re-Zahl Luftseite im turbulenten Bereich")
    xi_LRe = (1.8 * math.log10(Re_L) - 1.5)**(-2)
    Nu_L = ((xi_LRe / 8) * Re_L * Pr_L) / (1 + 12.7 * ((xi_LRe / 8)**(1 / 2)) * (Pr_L**(2 / 3) - 1))

alpha_L = Nu_L / (D_hydr_L / 1000) * lambda_L

Re-Zahl Luftseite im laminaren Bereich


##### 3.2 Rauchgasseite

In [ ]:
Kanalweite_R = 4  # mm
Kanalhoehe_R = 226  # mm
Kanallaenge_R = 1168  # mm
N_Blockreihen_R = 1  # -
N_Parallelkanaele_R = 57  # -
N_Ebenen_R = 1  # -
Umittel_R = V_R_real / 3600 / (Kanalweite_R * 0.001) / (Kanalhoehe_R * 0.001) / N_Parallelkanaele_R / N_Ebenen_R  # m/s
D_hydr_R = 2 * Kanalweite_R  # mm
Re_R = Umittel_R * D_hydr_R / 1000 / nu_R  # -

if Re_R < Re_lam:
    print("Re-Zahl Rauchgasseite im laminaren Bereich")
    Gz_RRe = Re_R * Pr_R * D_hydr_R / Kanallaenge_R
    Nu1_RRe = 7.54
    Nu2_RRe = 1.849 * Gz_RRe**(1 / 3)
    Nu3_RRe = (2 / (1 + 22 * Pr_R))**(1 / 6) * Gz_RRe**(1 / 2)
    Nu_R = (Nu1_RRe**3 + Nu2_RRe**3 + Nu3_RRe**3)**(1 / 3)
elif Re_R < Re_turb:
    print("Re-Zahl Rauchgasseite im Übergangsbereich")
    Gz_R2300 = Re_lam * Pr_R * D_hydr_R / Kanallaenge_R
    Nu1_R2300 = 7.54
    Nu2_R2300 = 1.849 * Gz_R2300**(1 / 3)
    Nu3_R2300 = (2 / (1 + 22 * Pr_R))**(1 / 6) * Gz_R2300**(1 / 2)
    Nu_R2300 = (Nu1_R2300**3 + Nu2_R2300**3 + Nu3_R2300**3)**(1 / 3)

    xi_R10000 = (1.8 * math.log10(Re_turb) - 1.5)**(-2)
    Nu_R10000 = ((xi_R10000 / 8) * Re_turb * Pr_R) / (1 + 12.7 * ((xi_R10000 / 8)**(1 / 2)) * (Pr_R**(2 / 3) - 1))
    gamma_R = (Re_R - Re_lam) / (Re_turb - Re_lam)
    Nu_R = (1 - gamma_R) * Nu_R2300 + gamma_R * Nu_R10000
else:
    print("Re-Zahl Rauchgasseite im turbulenten Bereich")
    xi_RRe = (1.8 * math.log10(Re_R) - 1.5)**(-2)
    Nu_R = ((xi_RRe / 8) * Re_R * Pr_R) / (1 + 12.7 * ((xi_RRe / 8)**(1 / 2)) * (Pr_R**(2 / 3) - 1))

alpha_R = Nu_R / (D_hydr_R / 1000) * lambda_R  # W/m^2/K

Re-Zahl Rauchgasseite im laminaren Bereich


##### 3.3 Wärmedurchgangskoeffizient k und Wärmeübertragungsfähigkeit kA

In [ ]:
staerke_Wand = 0.3  # mm
lambda_Wand = 15  # W/m/K
k = 1 / (1 / alpha_L + (staerke_Wand / 1000 / lambda_Wand) + 1 / alpha_R)  # W/m^2/K
A = 2 * N_Kanaele_L * N_Ebenen_L * N_Blockreihen_L * (Kanallaenge_L / 1000) * (Kanalhoehe_L / 1000)  # m^2
kA = k * A / 1000  # kW/K
print(f"kA = {kA:.2f} kW/K")

kA = 0.60 kW/K
